In [5]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Load the .mat file
data = sio.loadmat('data/gait_recording_080624_walk.mat')

In [6]:
# Extract required variables from the .mat file
t_insole_l = data['t_insole_l'].flatten()
t_trackers = data['t_trackers'].flatten()
jnt_pos_all_l = data['jnt_pos_all_l']
jnt_pos_all_r = data['jnt_pos_all_r']
jnt_angles_all_l = data['jnt_angles_all_l']
jnt_angles_all_r = data['jnt_angles_all_r']
insoleAll_l = data['insoleAll_l']
insoleAll_r = data['insoleAll_r']

color_r = 'red'
color_l = 'blue'

In [ ]:
# Set the start and stop indices for the animation
start_idx = 316
stop_idx = 544
start_t = t_insole_l[start_idx]
stop_t = t_insole_l[stop_idx]
maxLineLen = 100  # Number of samples displayed by the animated lines

# Create the figure and subplots
fig, axs = plt.subplots(3, 9, figsize=(15, 10))
fig.suptitle('Gait Animation')

# Define axes for different plots
p1 = axs[0, 0:2]
p2 = axs[0, 2:4]
p3 = axs[0, 4:6]
p4 = axs[0, 6]
p5 = axs[0, 7]
p6 = axs[0, 8]
p7 = axs[1, 6]
p8 = axs[1, 7]
p9 = axs[1, 8]
p10 = axs[2, 6]
p11 = axs[2, 7]
p12 = axs[2, 8]

# Initialize animated lines for each subplot
lines = {}
for idx, ax in enumerate([p4, p5, p6, p7, p8, p9, p10, p11, p12]):
    ax.set_ylim([-90, 90])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Angle (deg)')
    if idx == 0:
        ax.set_title('Hip Adduction/Abduction')
    elif idx == 1:
        ax.set_title('Hip Internal/External')
    elif idx == 2:
        ax.set_title('Hip Flexion/Extension')
    elif idx == 3:
        ax.set_title('Knee Adduction/Abduction')
    elif idx == 4:
        ax.set_title('Knee Internal/External')
    elif idx == 5:
        ax.set_title('Knee Flexion/Extension')
    elif idx == 6:
        ax.set_title('Ankle Inversion/Eversion')
    elif idx == 7:
        ax.set_title('Ankle Adduction/Abduction')
    elif idx == 8:
        ax.set_title('Ankle Dorsi/Plantar Flexion')
    
    # Store animated lines
    lines[idx] = {'line_r': ax.plot([], [], color=color_r, lw=2)[0], 'line_l': ax.plot([], [], color=color_l, lw=2)[0]}

# Initialize plot limits for joint positions
lim_x, lim_y, lim_z = [-1.11, -0.36], [0.06, 1.31], [0.06, 1.31]  # Adjust these limits as needed

def animate(i):
    if i >= len(t_insole_l):
        return

    idx = np.argmin(np.abs(t_trackers - t_insole_l[i]))
    
    pos_l = jnt_pos_all_l[idx].reshape(-1, 3)
    pos_r = jnt_pos_all_r[idx].reshape(-1, 3)
    jnt_angles_l = jnt_angles_all_l[idx].reshape(-1, 3)
    jnt_angles_r = jnt_angles_all_r[idx].reshape(-1, 3)
    
    # Update animated lines for each subplot
    for k in range(len(lines)):
        if k < 3:
            angle_index = k  # Corresponding angle indices for hips
        else:
            angle_index = k - 3  # Corresponding angle indices for knees and ankles
        lines[k]['line_r'].set_data(t_trackers[:idx], jnt_angles_r[:, angle_index])
        lines[k]['line_l'].set_data(t_trackers[:idx], jnt_angles_l[:, angle_index])
    
    # Update insole heatmaps
    if 'insoleAll_l' in locals():
        img_l = insoleAll_l[i, :].reshape(64, 16)
        p2.imshow(img_l, cmap='hot', aspect='auto')
        p2.set_title("Left Insole")
    
    if 'insoleAll_r' in locals():
        img_r = np.fliplr(insoleAll_r[i, :].reshape(64, 16))
        img_r[:32, :] = np.flipud(img_r[:32, :])
        p3.imshow(img_r, cmap='hot', aspect='auto')
        p3.set_title("Right Insole")

    return list(lines[k]['line_r'] for k in range(len(lines))) + list(lines[k]['line_l'] for k in range(len(lines)))

# Animate the plot
ani = FuncAnimation(fig, animate, frames=range(start_idx, stop_idx), interval=50, blit=True)

plt.show()